In [1]:
import numpy as np
import pandas as pd
import re
import openpyxl

In [2]:
infile = open('airport-data-dictionaries.xlsx', mode='r+b')

In [3]:
df = pd.read_excel(infile, sheet_name=2, index_col='Field', usecols='A,C')

pd.set_option('display.max_columns', 5)
pd.set_option('display.max_rows', 140)
pd.set_option('display.max_colwidth', None)
df.head()

,Description
Field,
SiteNumber,"Landing facility site number. The unique identifying number of the airport whose runway is being described. Together with the runway id field, this provides the unique key to a runway record."
State,Runway state post office code. The state where the landing facility is located. This was used in sorting the entire file by state and site number.
RunwayID,Runway identification. Ex. 01/19; 18L/36R (parallel runways); H1 (helipad); N/S (north/south); ALL/WAY (sealane); B1 (balloonport)
RunwayLength,Physical runway length (nearest foot) (ex. 3500)
RunwayWidth,Physical runway width (nearest foot) (ex. 100)


In [4]:
df.index.rename('category', inplace=True)

In [5]:
df.rename(columns={'Description': 'description'}, inplace=True)
df.head()

,description
category,
SiteNumber,"Landing facility site number. The unique identifying number of the airport whose runway is being described. Together with the runway id field, this provides the unique key to a runway record."
State,Runway state post office code. The state where the landing facility is located. This was used in sorting the entire file by state and site number.
RunwayID,Runway identification. Ex. 01/19; 18L/36R (parallel runways); H1 (helipad); N/S (north/south); ALL/WAY (sealane); B1 (balloonport)
RunwayLength,Physical runway length (nearest foot) (ex. 3500)
RunwayWidth,Physical runway width (nearest foot) (ex. 100)


In [6]:
#NOTE: 'ActivationDate' was modified to 'ActiviationDate' to match typo in source file. 

labels_to_keep = ['SiteNumber', 'State', 'RunwayID', 'RunwayLength', 'RunwayWidth', 'RunwaySurfaceTypeCondition', 'RunwaySurfaceTreatment', 'EdgeLightsIntensity', 'BaseEndID', 'BaseEndTrueAlignment', 'BaseEndILSType', 'BaseEndRightTrafficPattern', 'BaseEndMarkingsType', 'BaseEndMarkingsCondition', 'BaseEndPhysicalLatitude', 'BaseEndPhysicalLatitudeS', 'BaseEndPhysicalLongitude', 'BaseEndPhysicalLongitudeS', 'BaseEndPhysicalElevation', 'BaseEndCrossingHeight', 'BaseEndGlidePathAngle', 'BaseEndDisplacedLength', 'BaseEndVASI', 'BaseEndCenterlineLights', 'BaseEndTouchdownLights', 'BaseEndObjectDescription', 'BaseEndObjectHeight', 'ReciprocalEndID', 'ReciprocalEndTrueAlignment', 'ReciprocalEndILSType', 'ReciprocalEndRightTrafficPattern', 'ReciprocalEndMarkingsType', 'ReciprocalEndMarkingsCondition', 'ReciprocalEndPhysicalLatitude', 'ReciprocalEndPhysicalLatitudeS', 'ReciprocalEndPhysicalLongitude', 'ReciprocalEndPhysicalLongitudeS', 'ReciprocalEndPhysicalElevation', 'ReciprocalEndCrossingHeight', 'ReciprocalEndGlidePathAngle', 'ReciprocalEndDisplacedLength', 'ReciprocalEndVASI', 'ReciprocalEndCenterlineLights', 'ReciprocalEndTouchdownLights', 'ReciprocalEndObjectDescription', 'ReciprocalEndObjectHeight', 'RunwayWeightBearingCapacitySW', 'RunwayWeightBearingCapacityDW', 'RunwayWeightBearingCapacityDT', 'RunwayWeightBearingCapacityDDT', 'BaseEndTakeOffRunAvailableTORA', 'BaseEndTakeOffDistanceAvailableTODA', 'BaseEndAcltStopDistanceAvailableASDA', 'BaseEndLandingDistanceAvailableLDA', 'ReciprocalEndTakeOffRunAvailableTORA', 'ReciprocalEndTakeOffDistanceAvailableTODA', 'ReciprocalEndAcltStopDistanceAvailableASDA', 'ReciprocalEndLandingDistanceAvailableLDA']

labels_to_drop = [x for x in list(df.index) if x not in labels_to_keep]


In [7]:
df.drop(index=labels_to_drop, inplace=True)
df.head()

,description
category,
SiteNumber,"Landing facility site number. The unique identifying number of the airport whose runway is being described. Together with the runway id field, this provides the unique key to a runway record."
State,Runway state post office code. The state where the landing facility is located. This was used in sorting the entire file by state and site number.
RunwayID,Runway identification. Ex. 01/19; 18L/36R (parallel runways); H1 (helipad); N/S (north/south); ALL/WAY (sealane); B1 (balloonport)
RunwayLength,Physical runway length (nearest foot) (ex. 3500)
RunwayWidth,Physical runway width (nearest foot) (ex. 100)


In [8]:
index_labels = df.index.values


In [9]:
index_labels

array(['SiteNumber', 'State', 'RunwayID', 'RunwayLength', 'RunwayWidth',
       'RunwaySurfaceTypeCondition', 'RunwaySurfaceTreatment',
       'EdgeLightsIntensity', 'BaseEndID', 'BaseEndTrueAlignment',
       'BaseEndILSType', 'BaseEndRightTrafficPattern',
       'BaseEndMarkingsType', 'BaseEndMarkingsCondition',
       'BaseEndPhysicalLatitude', 'BaseEndPhysicalLatitudeS',
       'BaseEndPhysicalLongitude', 'BaseEndPhysicalLongitudeS',
       'BaseEndPhysicalElevation', 'BaseEndCrossingHeight',
       'BaseEndGlidePathAngle', 'BaseEndDisplacedLength', 'BaseEndVASI',
       'BaseEndCenterlineLights', 'BaseEndTouchdownLights',
       'BaseEndObjectDescription', 'BaseEndObjectHeight',
       'ReciprocalEndID', 'ReciprocalEndTrueAlignment',
       'ReciprocalEndILSType', 'ReciprocalEndRightTrafficPattern',
       'ReciprocalEndMarkingsType', 'ReciprocalEndMarkingsCondition',
       'ReciprocalEndPhysicalLatitude', 'ReciprocalEndPhysicalLatitudeS',
       'ReciprocalEndPhysicalLongitude',

In [10]:
for i, label in enumerate(index_labels):
    index_labels[i] = re.sub(r'(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|(?<=[A-Z])(?=ID)', '_', label)
    index_labels[i] = label.strip().lower()
    index_labels[i] = re.sub(r'_end', '', label)
    index_labels[i] = re.sub(r'reciprocal', 'rcpr', label)
    index_labels[i] = re.sub(r'rcpr_id', 'reciprocal_end_id', label)
    index_labels[i] = re.sub(r'base_id', 'base_end_id', label)
    index_labels[i] = re.sub(r'condition', 'cond', label)
    index_labels[i] = re.sub(r'_physical|_description|_traffic|', '', label)
    index_labels[i] = re.sub(r'weight_bearing_capacity', 'capacity_lbs', label)
    index_labels[i] = re.sub(r'_take_off_run_available|_take_off_distance_available|_aclt_stop_distance_available|_landing_distance_available',
                           '', label)

In [11]:
df

,description
category,
SiteNumber,"Landing facility site number. The unique identifying number of the airport whose runway is being described. Together with the runway id field, this provides the unique key to a runway record."
State,Runway state post office code. The state where the landing facility is located. This was used in sorting the entire file by state and site number.
RunwayID,Runway identification. Ex. 01/19; 18L/36R (parallel runways); H1 (helipad); N/S (north/south); ALL/WAY (sealane); B1 (balloonport)
RunwayLength,Physical runway length (nearest foot) (ex. 3500)
RunwayWidth,Physical runway width (nearest foot) (ex. 100)
RunwaySurfaceTypeCondition,"Runway surface type and condition. CONC - Portland cement concrete; ASPH - asphalt or bituminous concrete; SNOW - snow; ICE - ice; MATS - pierced steel planking, landing mats, membranes; TREATED - oiled, soil cement or lime stabilized; GRAVEL - gravel, cinders, crushed rock, coral or shells, slag; TURF - grass, sod; DIRT - natural soil; WATER - water; E - excellent condition; G - good condition; F - fair condition; P - poor condition; L - failed condition"
RunwaySurfaceTreatment,Runway surface treatment. (ex. GRVD - saw-cut or plastic grooved; PFC - porous friction course; AFSC - aggregate friction seal coat; RFSC - rubberized friction seal coat; WC - wire comb or wire tine; NONE - no special surface treatment)
EdgeLightsIntensity,Runway lights edge intensity. (ex. HIGH - high; MED - medium; LOW - low; NSTD - non-standard lighting system; NONE - no edge lighting system)
BaseEndID,Base end identifier. The runway end described by the following information. Relates to the first part of the runway identification. (ex. For runway 18/36 the base end is 18)


In [12]:
infile.close()

In [26]:
outfile = open('../Clean-Data/data_dictionary.xlsx', mode='r+b')
df_facs = pd.read_excel(outfile, sheet_name='Facilities', index_col='category')
df_facs.head()
outfile.close()

In [27]:
with pd.ExcelWriter('../Clean-Data/data_dictionary.xlsx') as writer:
    df_facs.to_excel(writer, sheet_name='Facilites', index_label='category')
    df.to_excel(writer, sheet_name='Runways', index_label='category')
    